# Test-02
Ejecutado en **Google Collaboratory**.

## Descripción
Se buscó probar técnicas de Transfer Learning utilizando la arquitectura VGG16 con sus coeficientes o parámetros ya entrenados y congelados o bien haciendo un ajuste fino a los mismos para adaptarlos al conjunto de datos de CIFAR-100. La conclusión fue que no se pudo obtener una buena métrica, y que había overfitting por la poca cantidad de datos para reentrenar toda la red. Esto motiva incursionar en técnicas de data augmentation para poder seguir con pruebas de esta magnitud.

In [1]:
import numpy as np

In [2]:
import pandas as pd

## Cargando las bases de datos

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
x_train_valid = np.load('drive/Shareddrives/Redes Neuronales BIO/CIFAR-100/x_train.npy')
y_train_valid = np.load('drive/Shareddrives/Redes Neuronales BIO/CIFAR-100/y_train.npy')
x_test = np.load('drive/Shareddrives/Redes Neuronales BIO/CIFAR-100/x_test.npy')

# Separando conjuntos para entrenamiento, validación

In [5]:
from sklearn.model_selection import train_test_split

In [20]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_valid, y_train_valid, test_size=0.1, random_state=25, stratify=y_train_valid)

# Normalización de datos

In [21]:
x_train_norm = x_train / 255
x_valid_norm = x_valid / 255
x_test_norm = x_test / 255

# Modelos

In [22]:
from keras.layers import Dense, Flatten, Activation, BatchNormalization, Dropout
from keras.layers import Conv2D, MaxPooling2D, Input, InputLayer, AveragePooling2D
from keras.models import Sequential, Model
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.optimizers import Adam
from keras.regularizers import l2

In [23]:
from keras.applications.vgg16 import VGG16

In [24]:
import keras

## Modelo #1

In [25]:
# Create an instance of the VGG16 to use Transfer Learning and disable the trainable option
preproc = VGG16(include_top=False, weights='imagenet', input_shape=(32, 32, 3))
for layer in preproc.layers:
        layer.trainable = False

# Preprocessing to extract valuable features from input images
x_train_pre = preproc.predict(x_train_norm)
x_valid_pre = preproc.predict(x_valid_norm)

In [36]:
# Create layers
input_layer = Input(shape=x_train_pre[0].shape)
layer = Flatten()(input_layer)
layer = Dense(units=256)(layer)
layer = BatchNormalization()(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.4)(layer)
layer = Dense(units=256)(layer)
layer = BatchNormalization()(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.4)(layer)
layer = Dense(units=100)(layer)
layer = BatchNormalization()(layer)
output_layer = Activation('softmax')(layer)

# Create model
model = Model(input_layer, output_layer)

# Compile
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy']
             )

In [37]:
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 1, 1, 512)]       0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               131328    
_________________________________________________________________
batch_normalization_11 (Batc (None, 256)               1024      
_________________________________________________________________
activation_11 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 256)               6579

In [38]:
# Create the ModelCheckpoint callback to save the best model during training
mc_callback = ModelCheckpoint('model_1.hdf5',
                              monitor='val_accuracy',
                              save_best_only=True,
                              verbose=0,
                              mode='max'
                             )

# Train the model
model.fit(x_train_pre, y_train,
          validation_data=(x_valid_pre, y_valid),
          callbacks=[mc_callback],
          epochs=30,
          batch_size=256
         )

Epoch 1/30
176/176 [==============================] - 3s 7ms/step - loss: 4.3342 - accuracy: 0.0723 - val_loss: 3.6543 - val_accuracy: 0.2146
Epoch 2/30
176/176 [==============================] - 1s 5ms/step - loss: 3.4654 - accuracy: 0.1958 - val_loss: 3.2482 - val_accuracy: 0.2632
Epoch 3/30
176/176 [==============================] - 1s 5ms/step - loss: 3.2442 - accuracy: 0.2326 - val_loss: 3.0694 - val_accuracy: 0.2908
Epoch 4/30
176/176 [==============================] - 1s 5ms/step - loss: 3.0983 - accuracy: 0.2594 - val_loss: 2.9623 - val_accuracy: 0.3044
Epoch 5/30
176/176 [==============================] - 1s 5ms/step - loss: 2.9964 - accuracy: 0.2779 - val_loss: 2.8662 - val_accuracy: 0.3096
Epoch 6/30
176/176 [==============================] - 1s 5ms/step - loss: 2.9258 - accuracy: 0.2887 - val_loss: 2.8010 - val_accuracy: 0.3210
Epoch 7/30
176/176 [==============================] - 1s 5ms/step - loss: 2.8652 - accuracy: 0.3013 - val_loss: 2.7462 - val_accuracy: 0.3288
Epoch 

In [16]:
# Load the model and show the final metrics
model = keras.models.load_model('model_1.hdf5')

# Train and validation metrics
_, train_acc = model.evaluate(x_train_pre, y_train, verbose=0)
_, valid_acc = model.evaluate(x_valid_pre, y_valid, verbose=0)

# Show result
print(f'[Accuracy] Train: {round(train_acc, 3)} Valid: {round(valid_acc, 3)}')

[Accuracy] Train: 0.566 Valid: 0.359


## Modelo #2


In [39]:
# Creating the total or global model
total_model_output = model(preproc.output)
total_model = Model(preproc.input, total_model_output)

# Enabling the VGG16 parameter training
for layer in preproc.layers:
    layer.trainable=True

# Compile
total_model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy']
             )

In [40]:
total_model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

In [41]:
# Create the ModelCheckpoint callback to save the best model during training
mc_callback = ModelCheckpoint('model_2.hdf5',
                              monitor='val_accuracy',
                              save_best_only=True,
                              verbose=0,
                              mode='max'
                             )

# Train the model
total_model.fit(x_train_norm, y_train,
          validation_data=(x_valid_norm, y_valid),
          callbacks=[mc_callback],
          epochs=30,
          batch_size=512
         )

Epoch 1/30
87/88 [============================>.] - ETA: 0s - loss: 5.0334 - accuracy: 0.0379

KeyboardInterrupt: ignored